In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_regression
import re
from sklearn.feature_selection import SelectKBest, f_classif, f_regression, mutual_info_regression
from sklearn.feature_selection import RFECV, RFE
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import shap
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import optuna
from metrics import *
from xgboost import XGBClassifier, XGBRegressor
from scipy.spatial import distance

/home/kamila/.pyenv/versions/3.9.0/lib/python3.9/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [2]:
df_original = pd.read_csv('../tabelas/dados_final.csv', encoding='ISO-8859-1', sep=";")

In [3]:
#pd.set_option('display.max_columns', None)
df_original

,ALUNOS_ID_ESCOLA,ALUNOS_PORCENTAGEM_Q002_Branca,ALUNOS_PORCENTAGEM_Q002_Preta,ALUNOS_PORCENTAGEM_Q002_Parda,ALUNOS_PORCENTAGEM_Q002_Amarela,ALUNOS_PORCENTAGEM_Q002_Indigena,ALUNOS_PORCENTAGEM_Q002_Nao_Declarado,ALUNOS_PORCENTAGEM_Q003A_Nao_moro_com_mae_madrasta,ALUNOS_PORCENTAGEM_Q003A_Sim_moro_com_mae_madrasta,ALUNOS_PORCENTAGEM_Q003B_Nao_moro_com_pai_padrasto,...,MEDIA_EM_LP,MEDIA_EM_MT,Escola_Federal,Escola_Estadual,Escola_Municipal,Escola_Privada,Capital_Area,Interior_Area,Regiao_Urbana,Regiao_Rural
0,11024968,0.303030,0.075758,0.575758,0.000000,0.015152,0.000000,0.121212,0.833333,0.287879,...,306.34,313.39,False,True,False,False,False,True,True,False
1,11025638,0.225352,0.049296,0.654930,0.007042,0.014085,0.014085,0.147887,0.732394,0.218310,...,266.00,272.43,False,True,False,False,False,True,True,False
2,11007168,0.178571,0.285714,0.464286,0.000000,0.035714,0.000000,0.178571,0.821429,0.500000,...,275.19,272.74,False,True,False,False,False,True,True,False
3,11007885,0.353333,0.066667,0.500000,0.046667,0.013333,0.000000,0.146667,0.786667,0.266667,...,302.97,301.57,False,True,False,False,False,True,True,False
4,11007893,0.258929,0.098214,0.531250,0.044643,0.000000,0.013393,0.120536,0.812500,0.263393,...,279.15,278.61,False,True,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10669,53014308,0.149254,0.164179,0.522388,0.074627,0.014925,0.000000,0.089552,0.791045,0.253731,...,298.54,298.90,False,True,False,False,True,False,True,False
10670,53014316,0.120000,0.200000,0.480000,0.060000,0.000000,0.100000,0.140000,0.760000,0.420000,...,276.98,262.95,False,True,False,False,True,False,True,False
10671,53051009,0.247423,0.113402,0.515464,0.030928,0.020619,0.030928,0.144330,0.783505,0.360825,...,300.33,294.99,False,True,False,False,True,False,True,False
10672,53068068,0.180328,0.169399,0.612022,0.005464,0.000000,0.027322,0.087432,0.885246,0.234973,...,284.52,274.22,False,True,False,False,True,False,True,False


# Removendo excesso de 0 no dataset

In [4]:
# Copiar para um novo dataframe de processamento
df_original["MEDIA_FINAL"] = df_original[["MEDIA_EM_LP", "MEDIA_EM_MT"]].mean(axis=1)
df_original2 = df_original.drop(columns = ['MEDIA_EM_LP', 'MEDIA_EM_MT'])
df_processado = df_original2.copy()


In [5]:
print("\n--- Etapa 2: Filtrando linhas com dados de questionário insuficientes (lógica geral aprimorada) ---")
    
# Identificar todas as colunas de porcentagem dos questionários
questionario_cols = [col for col in df_processado.columns if 'PORCENTAGEM' in col]

# Garantir que as colunas são numéricas para a comparação
for col in questionario_cols:
    df_processado[col] = pd.to_numeric(df_processado[col], errors='coerce')

print(f"Total de colunas de questionário consideradas para o filtro: {len(questionario_cols)}")

# Calcular a porcentagem de colunas zeradas para cada linha
percentual_zeros = (df_processado[questionario_cols] == 0).mean(axis=1)

# --- Análise Diagnóstica (para ajudar a definir o limite) ---
print("\nAnálise da distribuição de linhas com valores zero:")
print(f" - {(percentual_zeros > 0.95).sum()} linhas têm mais de 95% de zeros.")
print(f" - {(percentual_zeros > 0.90).sum()} linhas têm mais de 90% de zeros.")
print(f" - {(percentual_zeros > 0.80).sum()} linhas têm mais de 80% de zeros.")
print(f" - {(percentual_zeros > 0.70).sum()} linhas têm mais de 70% de zeros.")
print(f" - {(percentual_zeros > 0.67).sum()} linhas têm mais de 67% de zeros.")
print(f" - {(percentual_zeros > 0.65).sum()} linhas têm mais de 65% de zeros.")
print(f" - {(percentual_zeros > 0.60).sum()} linhas têm mais de 60% de zeros.")

# Definir um limite: removeremos linhas se mais de 80% das colunas de questionário forem 0.
# Este valor é mais rigoroso que os 95% anteriores e pode precisar de ajuste.
LIMITE_ZEROS = 0.65
print(f"\nDefinindo o limite de remoção em >{LIMITE_ZEROS*100}% de valores zero.")

# Manter apenas as linhas que estão ABAIXO do limite de zeros
linhas_a_manter = percentual_zeros <= LIMITE_ZEROS

linhas_removidas = df_processado.shape[0] - linhas_a_manter.sum()
df_processado = df_processado[linhas_a_manter].copy()

print(f"\nLinhas removidas: {linhas_removidas}")
print(f"Tamanho do dataset após filtragem: {df_processado.shape[0]} linhas.")


--- Etapa 2: Filtrando linhas com dados de questionário insuficientes (lógica geral aprimorada) ---
Total de colunas de questionário consideradas para o filtro: 1324

Análise da distribuição de linhas com valores zero:
 - 0 linhas têm mais de 95% de zeros.
 - 0 linhas têm mais de 90% de zeros.
 - 0 linhas têm mais de 80% de zeros.
 - 281 linhas têm mais de 70% de zeros.
 - 1330 linhas têm mais de 67% de zeros.
 - 2525 linhas têm mais de 65% de zeros.
 - 6787 linhas têm mais de 60% de zeros.

Definindo o limite de remoção em >65.0% de valores zero.

Linhas removidas: 2525
Tamanho do dataset após filtragem: 8149 linhas.


In [8]:
print("\n--- Etapa 3: Selecionando a melhor feature por grupo de pergunta ---")
mapeamento_renomeacao = {}
colunas_finais_selecionadas = []

def selecionar_e_renomear_features(df, prefixo):
    """
    Seleciona a melhor coluna de cada grupo de perguntas com base na Informação Mútua
    e gera um nome curto e significativo para ela.
    """
    #nonlocal mapeamento_renomeacao
    #nonlocal colunas_finais_selecionadas
    
    padrao_pergunta = re.compile(f"({prefixo}_PORCENTAGEM_Q\\d{{3}}[A-Z]?)_")
    colunas_prefixo = [col for col in df.columns if col.startswith(prefixo)]
    
    grupos_perguntas = {}
    for col in colunas_prefixo:
        match = padrao_pergunta.match(col)
        if match:
            prefixo_pergunta = match.group(1)
            if prefixo_pergunta not in grupos_perguntas:
                grupos_perguntas[prefixo_pergunta] = []
            grupos_perguntas[prefixo_pergunta].append(col)

    y = df["MEDIA_FINAL"]
    
    for nome_grupo, colunas_grupo in grupos_perguntas.items():
        if not colunas_grupo:
            continue

        X_grupo = df[colunas_grupo].fillna(0)
        mi_scores = mutual_info_regression(X_grupo, y, random_state=42)
        
        melhor_coluna_original = colunas_grupo[np.argmax(mi_scores)]
        
        # Lógica para criar um nome mais curto e legível
        partes = melhor_coluna_original.split('_')
        # Ex: ALUNO_Q018A_Le_Noticias_Quase_Sempre -> A018A_le_noticias
        novo_nome = f"{partes[2]}_{partes[3].lower()}"
        
        # Adicionar ao mapeamento para referência futura
        mapeamento_renomeacao[novo_nome] = melhor_coluna_original
        
        # Adicionar a coluna original à lista de colunas a manter
        colunas_finais_selecionadas.append(melhor_coluna_original)

# Aplicar a função para cada grupo
selecionar_e_renomear_features(df_processado, 'ALUNOS')
selecionar_e_renomear_features(df_processado, 'PROFESSOR')
selecionar_e_renomear_features(df_processado, 'DIRETOR')

print(f"Total de {len(colunas_finais_selecionadas)} features selecionadas.")


--- Etapa 3: Selecionando a melhor feature por grupo de pergunta ---
Total de 396 features selecionadas.


In [11]:
# 4. CRIAÇÃO DO DATASET FINAL E SALVAMENTO
# ==============================================================================
print("\n--- Etapa 4: Criando e salvando o dataset final ---")

# Colunas de identificação e a variável alvo
colunas_essenciais = ["ALUNOS_ID_ESCOLA", "MEDIA_FINAL"]

# Criar o DataFrame final apenas com as colunas selecionadas e essenciais
df_final = df_processado[colunas_essenciais + colunas_finais_selecionadas].copy()

# Renomear as colunas para os nomes curtos
mapa_inverso = {v: k for k, v in mapeamento_renomeacao.items()}
df_final.rename(columns=mapa_inverso, inplace=True)
df_final.rename(columns={"ALUNOS_ID_ESCOLA": "ID_ESCOLA"}, inplace=True)

# Salvar o mapeamento de renomeação para um arquivo CSV para sua referência
df_mapeamento = pd.DataFrame(list(mapeamento_renomeacao.items()), columns=['Novo_Nome_Feature', 'Nome_Original_Feature'])
#df_mapeamento.to_csv('renomeacao_mapeamento.csv', index=False)
#print("Arquivo 'renomeacao_mapeamento.csv' com o dicionário de dados foi salvo.")




--- Etapa 4: Criando e salvando o dataset final ---


,Novo_Nome_Feature,Nome_Original_Feature
0,Q002_branca,ALUNOS_PORCENTAGEM_Q002_Branca
1,Q003A_sim,ALUNOS_PORCENTAGEM_Q003A_Sim_moro_com_mae_madr...
2,Q003B_sim,ALUNOS_PORCENTAGEM_Q003B_Sim_moro_com_pai_padr...
3,Q003C_sim,ALUNOS_PORCENTAGEM_Q003C_Sim_moro_com_irmaos
4,Q003D_nao,ALUNOS_PORCENTAGEM_Q003D_Nao_moro_com_avos
...,...,...
391,Q247_treinamento,DIRETOR_PORCENTAGEM_Q247_Treinamento_Sobre_Ceg...
392,Q248_treinamento,DIRETOR_PORCENTAGEM_Q248_Treinamento_Sobre_Bai...
393,Q249_treinamento,DIRETOR_PORCENTAGEM_Q249_Treinamento_Sobre_Def...
394,Q250_treinamento,DIRETOR_PORCENTAGEM_Q250_Treinamento_Sobre_Sup...


In [12]:
df_final

,ID_ESCOLA,MEDIA_FINAL,Q002_branca,Q003A_sim,Q003B_sim,Q003C_sim,Q003D_nao,Q003E_sim,Q004_mae,Q005_pai,...,Q242_treinamento,Q243_treinamento,Q244_treinamento,Q245_treinamento,Q246_treinamento,Q247_treinamento,Q248_treinamento,Q249_treinamento,Q250_treinamento,Q251_treinamento
1,11025638,269.215,0.225352,0.732394,0.661972,0.591549,0.654930,0.147887,0.119718,0.211268,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11007168,273.965,0.178571,0.821429,0.500000,0.642857,0.678571,0.142857,0.107143,0.607143,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0
3,11007885,302.270,0.353333,0.786667,0.640000,0.600000,0.606667,0.146667,0.053333,0.140000,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
4,11007893,278.880,0.258929,0.812500,0.651786,0.575893,0.687500,0.138393,0.102679,0.214286,...,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
6,11026197,248.260,0.153061,0.683673,0.428571,0.602041,0.683673,0.183673,0.204082,0.408163,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10668,53013840,280.915,0.226804,0.876289,0.659794,0.752577,0.484536,0.123711,0.061856,0.134021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10669,53014308,298.720,0.149254,0.791045,0.582090,0.746269,0.537313,0.164179,0.059701,0.164179,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10671,53051009,297.660,0.247423,0.783505,0.484536,0.649485,0.546392,0.288660,0.082474,0.257732,...,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0
10672,53068068,279.370,0.180328,0.885246,0.693989,0.825137,0.699454,0.103825,0.131148,0.234973,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
